# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


# Dataset loading

In [3]:
dataset_name='pokemon'

In [4]:
IMG_DIR = 'C:\\Users\\Khalid\\Documents\projects\\pokemon\DS06\\'
VAL_PCT = 10
VAL_FORMAT = 'png'

In [6]:
os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..')
print(os.getcwd())

In [7]:
from training.generators.file_image_generator import create_image_lists, get_generators

Using TensorFlow backend.


In [8]:
imgs_list = create_image_lists(
    image_dir=IMG_DIR, 
    validation_pct=VAL_PCT, 
    valid_imgae_formats=VAL_FORMAT
)

  DEBUG    | Looking for images in 'all'


  INFO     | 809 file found


In [9]:
inputs_shape= IMG_SIZE=(200, 200, 3)
BATCH_SIZE = 32
latent_dim = 32
intermediate_dim = 32

In [10]:
train_gen, test_gen = get_generators(
    images_list=imgs_list, 
    image_dir=IMG_DIR, 
    image_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    class_mode=None
)

  INFO     | Found 738 training files


  INFO     | Found 71 validation files


In [11]:
import tensorflow as tf

In [12]:
train_ds = tf.data.Dataset.from_generator(
    lambda: train_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)

test_ds = tf.data.Dataset.from_generator(
    lambda: test_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)


In [13]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [14]:
_instance_scale

1.0

In [15]:
import numpy as np
from collections.abc import Iterable

In [16]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [17]:
_outputs_shape

120000

# Model's Layers definition

In [18]:
menc_lays = [tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latent_dim)]

venc_lays = [tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latent_dim)]

dec_lays = [tf.keras.layers.Dense(units=latent_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [20]:
model_name = dataset_name+'LTVAAEsharp_diff'
recording_dir='recording'+sep_local+model_name

In [23]:
from training.latent_adversarial.autoencoders.VAAE import VAAE as AE

In [24]:
inputs_shape=IMG_SIZE

In [25]:
variables_params = \
[
    {
        'name': 'encoder_mean', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': menc_lays#enc_lays2#
    }

    ,
    
    {
        'name': 'encoder_logvar', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': venc_lays#enc_lays2#
    }

    ,
        {
        'name': 'generative', 
        'inputs_shape':latent_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays#dec_lays2#
    }
]

In [26]:
from utils.data_and_files.file_utils import create_if_not_exist

In [27]:
_restore = os.path.join(recording_dir, 'var_save_dir')

In [28]:
create_if_not_exist(_restore)
_restore

'..\\..\\recording\\pokemonVAAEsharp_diff\\var_save_dir'

In [29]:
#to restore trained model, set filepath=_restore

In [ ]:
from stats.traditional_adversarial_losses import \
    create_latent_adversarial_discriminator_real_losses, \
    create_latent_adversarial_discriminator_fake_losses, \

latent_adversarial_discriminator_losses = {
    'latent_adversarial_discriminator_real_outputs': create_latent_adversarial_discriminator_real_losses,
    'latent_adversarial_discriminator_fake_outputs': create_latent_adversarial_discriminator_fake_losses,
    'latent_adversarial_generator_fake_outputs': create_latent_adversarial_generator_fake_losses,
}

In [30]:
ae = AE( 
    adversarial_losses=latent_adversarial_discriminator_losses,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=latent_dim,
    batch_size=BATCH_SIZE,
    variables_params=variables_params, 
    filepath=None
    )

ResourceExhaustedError: OOM when allocating tensor with shape[640000,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]

In [26]:
#from evaluation.quantitive_metrics.psnr import psnr
#from evaluation.quantitive_metrics.ssmi import ssmi
from evaluation.quantitive_metrics.sharp_diff import sharp_diff


In [27]:
ae.compile(loss={'x_logits': sharp_diff})

Model: "pokemonVAEsharp_diff"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_logvar_inputs (InputLay [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
encoder_logvar (Model)          (None, 32)           20480496    encoder_logvar_inputs[0][0]      
__________________________________________________________________________________________________
tf_op_layer_mul (TensorFlowOpLa [(None, 32)]         0           encoder_logvar[1][0]             
__________________________________________________________________________________________________
tf_op_layer_Exp (TensorFlowOpLa [(None, 32)]         0           tf_op_layer_mul[0][0]            
_______________________________________________________________________________

# Callbacks

In [28]:
from training.callbacks.progress_bar import NotebookPrograssBar
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [29]:
progbar = NotebookPrograssBar(leave_outer=False)

In [30]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [31]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [32]:
csv_dir = os.path.join(recording_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'..\\..\\recording\\pokemonVAEsharp_diff\\csv_dir\\pokemonVAEsharp_diff.csv'

In [33]:
image_gen_dir = os.path.join(recording_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [34]:
sg = SampleGeneration(latent_shape=latent_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [35]:
import numpy as np

# Model Training

In [36]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=100,
    epochs=int(1e6), 
    verbose=0,
    callbacks=[progbar, es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=100
)

KeyError: 'val_loss'